# Capstone Neighbor Project
In this project , machine learning tools will be used with python

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

In [3]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [4]:
print ('Hello Capstone Project Course!')

Hello Capstone Project Course!


1-) Download List of Postal Codes of Canada from wikipedia as html

In [5]:
import requests
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url).content
df_list = pd.read_html(html)

2-) Get table of postal codes as data frame from html data

In [6]:
df = pd.DataFrame (df_list[0])

3-) Drop row which is "Not assigned" and changes not assigned values in neighbourhood column to value of borough column in the same row.

In [7]:
df = df[df.Borough != "Not assigned"]

df["Neighbourhood"][df.Neighbourhood == "Not assigned" ] = "Queen's Park" # there is only on case like this 


4-) Combine rows which Postcode values are same and separate them with comma.

In [8]:
df['Neighbourhood'] = df['Neighbourhood'] + ","
df = df.groupby(["Postcode","Borough"],as_index=False).sum()
df['Neighbourhood'] = df['Neighbourhood'].str[:-1]

In [9]:
df.shape

(103, 3)

In [10]:
url = 'https://cocl.us/Geospatial_data'
df_geo = pd.read_csv(url)


In [11]:
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
df["Latitude"] = df_geo["Latitude"]
df["Longitude"] = df_geo["Longitude"]
df.drop(df.index[0])
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [13]:
toronto_grouped = df.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Latitude,Longitude
0,"Adelaide,King,Richmond",43.650571,-79.384568
1,Agincourt,43.794200,-79.262029
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",43.815252,-79.284577
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437
4,"Alderwood,Long Branch",43.602414,-79.543484


In [14]:
toronto_grouped.shape

(103, 3)

## Let see the map of Toronto

In [15]:

address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [16]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_grouped['Latitude'], toronto_grouped['Longitude'], df['Borough'], toronto_grouped['Neighbourhood']):
    label = '{}, {}'.format(toronto_grouped, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Analyze Each Neighborhood

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            'X31B5BHABVQWGVBZBUT5PDMMZLGCLNMON0VRCMYKIXKP4CWC', 
            'MMJBXHK02R3RMC5GUR4J1YW1EGWBQHLMKCDK1A0LIVYDJHD0', 
            '20180604', 
            lat, 
            lng, 
            radius, 
            30)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
toronto_venues = getNearbyVenues(names=toronto_grouped['Neighbourhood'],
                                   latitudes=toronto_grouped['Latitude'],
                                   longitudes=toronto_grouped['Longitude']
                                  )


Adelaide,King,Richmond
Agincourt
Agincourt North,L'Amoreaux East,Milliken,Steeles East
Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown
Alderwood,Long Branch
Bathurst Manor,Downsview North,Wilson Heights
Bayview Village
Bedford Park,Lawrence Manor East
Berczy Park
Birch Cliff,Cliffside West
Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe
Brockton,Exhibition Place,Parkdale Village
Business Reply Mail Processing Centre 969 Eastern
CFB Toronto,Downsview East
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Cabbagetown,St. James Town
Caledonia-Fairbanks
Canada Post Gateway Processing Centre
Cedarbrae
Central Bay Street
Chinatown,Grange Park,Kensington Market
Christie
Church and Wellesley
Clairlea,Golden Mile,Oakridge
Clarks Corners,Sullivan,Tam O'Shanter
Cliffcrest,Cliffside,Scarborough Village West
Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park
Commerce 

In [19]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Adelaide,King,Richmond",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Adelaide,King,Richmond",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Adelaide,King,Richmond",0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,"Adelaide,King,Richmond",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
toronto_onehot.shape

(1331, 232)

In [21]:
toronto_grouped2 = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped2

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.033333,0.0,...,0.0,0.033333,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000
1,Agincourt,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000
4,"Alderwood,Long Branch",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000
5,"Bathurst Manor,Downsview North,Wilson Heights",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.0,0.000000,0.0,0.052632,0.000000,0.0000,0.000000,0.000000,0.0,0.000000
6,Bayview Village,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000
7,"Bedford Park,Lawrence Manor East",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.045455,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000
8,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.0,0.033333,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000
9,"Birch Cliff,Cliffside West",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000


In [22]:
toronto_grouped2.shape

(99, 232)

In [23]:
num_top_venues = 5

for hood in toronto_grouped2['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped2[toronto_grouped2['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
              venue  freq
0        Steakhouse  0.10
1  Asian Restaurant  0.07
2             Hotel  0.07
3       Coffee Shop  0.07
4              Café  0.07


----Agincourt----
            venue  freq
0  Breakfast Spot  0.25
1          Lounge  0.25
2  Clothing Store  0.25
3    Skating Rink  0.25
4          Museum  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                      venue  freq
0                      Park  0.67
1                Playground  0.33
2                    Museum  0.00
3  Mediterranean Restaurant  0.00
4             Metro Station  0.00


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                  venue  freq
0         Grocery Store  0.22
1        Sandwich Place  0.11
2  Fast Food Restaurant  0.11
3   Fried Chicken Joint  0.11
4            Beer Store  0.11


----Alderwood,Long Branch----
            venue  freq
0     Pizza Place   0.2
1       

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped2['Neighbourhood']

for ind in np.arange(toronto_grouped2.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped2.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Steakhouse,Café,Hotel,Coffee Shop,Asian Restaurant,Neighborhood,Pizza Place,Lounge,Smoke Shop,Speakeasy
1,Agincourt,Lounge,Breakfast Spot,Clothing Store,Skating Rink,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Department Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Drugstore
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Fast Food Restaurant,Fried Chicken Joint,Coffee Shop,Sandwich Place,Pizza Place,Beer Store,Pharmacy,Gay Bar,Gastropub
4,"Alderwood,Long Branch",Pizza Place,Gym,Pool,Athletics & Sports,Pub,Skating Rink,Coffee Shop,Pharmacy,Sandwich Place,Electronics Store


# Cluster Neighborhoods

In [26]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped2.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 2, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [57]:
# add clustering labels 
neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

toronto_merged = toronto_grouped

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged = toronto_merged.dropna()

toronto_merged # check the last columns!

,Neighbourhood,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",43.650571,-79.384568,1.0,Steakhouse,Café,Hotel,Coffee Shop,Asian Restaurant,Neighborhood,Pizza Place,Lounge,Smoke Shop,Speakeasy
1,Agincourt,43.794200,-79.262029,1.0,Lounge,Breakfast Spot,Clothing Store,Skating Rink,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",43.815252,-79.284577,2.0,Park,Playground,Department Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Drugstore
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437,1.0,Grocery Store,Fast Food Restaurant,Fried Chicken Joint,Coffee Shop,Sandwich Place,Pizza Place,Beer Store,Pharmacy,Gay Bar,Gastropub
4,"Alderwood,Long Branch",43.602414,-79.543484,1.0,Pizza Place,Gym,Pool,Athletics & Sports,Pub,Skating Rink,Coffee Shop,Pharmacy,Sandwich Place,Electronics Store
5,"Bathurst Manor,Downsview North,Wilson Heights",43.754328,-79.442259,1.0,Coffee Shop,Pizza Place,Fried Chicken Joint,Shopping Mall,Fast Food Restaurant,Diner,Sandwich Place,Supermarket,Sushi Restaurant,Middle Eastern Restaurant
6,Bayview Village,43.786947,-79.385975,1.0,Chinese Restaurant,Bank,Japanese Restaurant,Café,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
7,"Bedford Park,Lawrence Manor East",43.733283,-79.419750,1.0,Italian Restaurant,Pizza Place,Coffee Shop,Pharmacy,Butcher,Restaurant,Indian Restaurant,Pub,Café,Thai Restaurant
8,Berczy Park,43.644771,-79.373306,1.0,Farmers Market,Café,Beer Bar,Coffee Shop,Cocktail Bar,Seafood Restaurant,Concert Hall,Tea Room,Breakfast Spot,Bistro
9,"Birch Cliff,Cliffside West",43.692657,-79.264848,1.0,College Stadium,Skating Rink,General Entertainment,Café,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store


In [49]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color= rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Cluster 1

In [50]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
42,43.724766,Yoga Studio,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
56,43.636258,Baseball Field,Yoga Studio,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store


# Cluster 2

In [53]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,43.650571,Café,Hotel,Coffee Shop,Asian Restaurant,Neighborhood,Pizza Place,Lounge,Smoke Shop,Speakeasy
1,43.794200,Breakfast Spot,Clothing Store,Skating Rink,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
3,43.739416,Fast Food Restaurant,Fried Chicken Joint,Coffee Shop,Sandwich Place,Pizza Place,Beer Store,Pharmacy,Gay Bar,Gastropub
4,43.602414,Gym,Pool,Athletics & Sports,Pub,Skating Rink,Coffee Shop,Pharmacy,Sandwich Place,Electronics Store
5,43.754328,Pizza Place,Fried Chicken Joint,Shopping Mall,Fast Food Restaurant,Diner,Sandwich Place,Supermarket,Sushi Restaurant,Middle Eastern Restaurant
6,43.786947,Bank,Japanese Restaurant,Café,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
7,43.733283,Pizza Place,Coffee Shop,Pharmacy,Butcher,Restaurant,Indian Restaurant,Pub,Café,Thai Restaurant
8,43.644771,Café,Beer Bar,Coffee Shop,Cocktail Bar,Seafood Restaurant,Concert Hall,Tea Room,Breakfast Spot,Bistro
9,43.692657,Skating Rink,General Entertainment,Café,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
10,43.643515,Coffee Shop,Liquor Store,Beer Store,Shopping Plaza,Café,Convenience Store,Pet Store,Pizza Place,Park


# Cluster 3

In [54]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,43.815252,Playground,Department Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Drugstore
13,43.737473,Park,Yoga Studio,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
16,43.689026,Market,Women's Store,Fast Food Restaurant,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
41,43.685347,Convenience Store,Yoga Studio,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
64,43.728020,Swim School,Bus Line,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
68,43.689574,Tennis Court,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
74,43.753259,Food & Drink Shop,Fast Food Restaurant,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
76,43.679563,Playground,Building,Trail,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
91,43.653654,Park,River,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
96,43.706876,Yoga Studio,Dessert Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant


# Cluster 4 

In [55]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,43.650943,Yoga Studio,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant


# Cluster 5

In [56]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,43.784535,Yoga Studio,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
71,43.706748,Drugstore,Rental Car Location,Bar,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
